In [1]:
!python -m pip install --upgrade pip

In [2]:
%pip install pandas
%pip install geopy
%pip install pycountry
%pip install datetime

from datetime import datetime
import pandas as pd
import os as os
from geopy.geocoders import Nominatim
from countryinfo import CountryInfo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
percorso_csv = r"../csv"

if os.path.exists(percorso_csv):
    for file in os.listdir(percorso_csv):
        percorso_csv_file = os.path.join(percorso_csv, file)
        print(f"Il file {percorso_csv_file} esiste.")
else:
    print(f"Il file {percorso_csv_file} non è stato trovato.")

Il file ../csv\appearances.csv esiste.
Il file ../csv\clubs.csv esiste.
Il file ../csv\club_games.csv esiste.
Il file ../csv\competitions.csv esiste.
Il file ../csv\games.csv esiste.
Il file ../csv\game_events.csv esiste.
Il file ../csv\game_lineups.csv esiste.
Il file ../csv\players.csv esiste.
Il file ../csv\player_valuations.csv esiste.


In [4]:
df_appearances = pd.read_csv(percorso_csv + "/appearances.csv")
df_club_games = pd.read_csv(percorso_csv + "/club_games.csv")
df_clubs = pd.read_csv(percorso_csv + "/clubs.csv")
df_competitions = pd.read_csv(percorso_csv + "/competitions.csv")
df_game_events = pd.read_csv(percorso_csv + "/game_events.csv")
df_game_lineups = pd.read_csv(percorso_csv + "/game_lineups.csv")
df_games = pd.read_csv(percorso_csv + "/games.csv")
df_player_valuations = pd.read_csv(percorso_csv + "/player_valuations.csv")
df_players = pd.read_csv(percorso_csv + "/players.csv")


C:\Users\mikdr\AppData\Local\Temp\ipykernel_9912\1344331387.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_game_lineups = pd.read_csv(percorso_csv + "/game_lineups.csv")


In [5]:
#da game_lineups elimino la colonna number (che rappresenta i numeri di maglia) che ai fini dell'analisi è totalmente inutile
del df_game_lineups["number"]
print(df_game_lineups)

                         game_lineups_id  game_id  club_id             type  \
0       3b0a25b9c76eedd7f6faaec061abf671  3606208      338  starting_lineup   
1       cb6eeedb9f728570615d37b6fe7facaf  3606208      338  starting_lineup   
2       ef88b1c4d61884ed8a5b2640d05a2dcd  3606208      338  starting_lineup   
3       baa0d6827dab4fab07d8dc1604e720a7  3606208      338  starting_lineup   
4       1715ec342bf902522b69322a036a3f29  3606208      338  starting_lineup   
...                                  ...      ...      ...              ...   
140315  f62e8310b6b6183961db0c3b32b94ddf  4243187      828      substitutes   
140316  dd066717338d299138c100b5e9584364  4243187      828      substitutes   
140317  12b990a69f1ffdab9aefa42bfda7b2e3  4243187      828      substitutes   
140318  4b43f9ddc4689376f0d7e6151458918a  4243187      828      substitutes   
140319  e66695a529986579822ecae6c34746d9  4243187      828      substitutes   

        player_id        player_name  team_captain 

In [6]:
#da player posso eliminare nome e cognome che tanto esiste una colonna che li ha entrambi
elimino = ["first_name", "last_name"]
df_players.drop(columns = elimino, inplace = True)

In [7]:
#nella tabella players ho visto che se c'è position c'è anche la subposition, se invece la position è Missing allora subposition è null --> risolvo mettendo missing
for index, player in df_players.iterrows():
    if pd.isnull(player["sub_position"]):
        df_players.at[index, "sub_position"] = "Missing"

In [8]:
#tabella players dove manca foot mettiamo destro che è il più comune nella popolazione mondiale ed anche nei calciatori
for index, player in df_players.iterrows():
    if pd.isnull(player["foot"]):
        df_players.at[index, "foot"] = "right"


In [9]:
#tabella players dove non c'è l'altezza mettiamo la media del ruolo che ha quel giocatore
for index, player in df_players.iterrows():
    if pd.isnull(player["height_in_cm"]):
        df_players.at[index, "height_in_cm"] = df_players["height_in_cm"].where(df_players["position"] == player["position"]).mean().round()

In [10]:
#tabella players per la scadenza del contratto i valori nulli li sostituisco con una data di default
for index, player in df_players.iterrows():
    if pd.isnull(player["contract_expiration_date"]):
        df_players.at[index, "contract_expiration_date"] = "Missing"

In [11]:
#in players per il procuratore invece mettiamo sconosciuto
for index, player in df_players.iterrows():
    if pd.isnull(player["agent_name"]):
        df_players.at[index, "agent_name"] = "Sconosciuto"

In [12]:
#in players dove non c'è valore mercato lo sostituisco col valore minimo di quel ruolo poichè abbiamo visto che sono giocatori minori o ritirati
for index, player in df_players.iterrows():
    if pd.isnull(player["market_value_in_eur"]):
        df_players.at[index, "market_value_in_eur"] = df_players["market_value_in_eur"].where(df_players["position"] == player["position"]).min()
    if pd.isnull(player["highest_market_value_in_eur"]):
        df_players.at[index, "highest_market_value_in_eur"] = df_players["highest_market_value_in_eur"].where(df_players["position"] == player["position"]).min()

In [13]:
#in tabella players qui andiamo a riempire i null nei campi cit, country e citizenship con delle funzioni di geopy e counterinfo che ci fanno risalire allo stato/capitale dalla capitale/stato
def da_citta_a_stato(city):
    geolocator = Nominatim(user_agent="google") #user agent can be any user agent
    location = geolocator.geocode(city, language="en", timeout=None) #specified the language as some countries are in other lanaguages
    if location and hasattr(location, "address"):
        country = location.address.split(',')[-1].strip()
        return country
    else:
        return "Sconosciuto"
    
def valori_mancanti(player):
    if pd.isnull(player["country_of_citizenship"]) and pd.isnull(player["country_of_birth"]) and pd.isnull(player["city_of_birth"]):
        df_players.at[index, "country_of_birth"] = "Sconosciuto"
        df_players.at[index,"city_of_birth"] = "Sconosciuto"
        df_players.at[index,"country_of_citizenship"] = "Sconosciuto"
    else:
        if pd.isnull(player["country_of_citizenship"]):
            df_players.at[index,"country_of_citizenship"] = "Sconosciuto"

        if pd.isnull(player["country_of_birth"]) and pd.isnull(player["city_of_birth"]):
            df_players.at[index,"country_of_birth"] = "Sconosciuto"
            df_players.at[index,"city_of_birth"] = "Sconosciuto"
        else:
            if pd.isnull(player["country_of_birth"]) and not pd.isnull(player["city_of_birth"]):
                df_players.at[index,"country_of_birth"] = da_citta_a_stato(player["city_of_birth"])
            else:
                if pd.isnull(player["city_of_birth"]) and not pd.isnull(player["country_of_birth"]):
                    df_players.at[index,"city_of_birth"] = CountryInfo(player["country_of_birth"]).capital()

for index, player in df_players.iterrows():
    valori_mancanti(player)

In [14]:
#tabella games risolviamo la mancanza della posizione in classifica mettendo la mediana E' DA VALUTARE POICHè MANCA IL 30% DEI DATI
mediana = df_games["home_club_position"].median()

for index, game in df_games.iterrows():
    if pd.isnull(game["home_club_position"]):
        df_games.at[index, "home_club_position"] = mediana
    if pd.isnull(game["away_club_position"]):
        df_games.at[index, "away_club_position"] = mediana
            

In [15]:
#in games risolviamo mancanza spettatori mettendo la media di tutte le partite
media = df_games["attendance"].mean().round()

for index, game in df_games.iterrows():
    if pd.isnull(game["attendance"]):
        df_games.at[index, "attendance"] = media

In [16]:
#in games quando arbitro non c'è metto sconosciuto

for index, game in df_games.iterrows():
    if pd.isnull(game["referee"]):
        df_games.at[index, "referee"] = "Sconosciuto"
    

In [17]:
#elimino le colonne home/away formation perchè manca nel 95% TABELLA: games
cancella = ["home_club_formation", "away_club_formation"]
df_games.drop(columns = cancella, inplace = True)

In [18]:
#se non ci sta il nome dei club su TABELLA: game vuol dire che manca l'id in TABELLA: club (vincolo non rispettato) per cui elimino la riga
for index, game in df_games.iterrows():
    if pd.isnull(game["home_club_name"]) or pd.isnull(game["away_club_name"]):
        df_games.drop(index, inplace=True)

KeyboardInterrupt: 

In [ ]:
#tabella games se manca allenatore metto sconosciuto
for index, game in df_games.iterrows():
    if pd.isnull(game["home_club_manager_name"]):
        df_games.at[index, "home_club_manager_name"] = "Sconosciuto"
    if pd.isnull(game["away_club_manager_name"]):
        df_games.at[index, "away_club_manager_name"] = "Sconosciuto"

      home_club_manager_name away_club_manager_name
0       Stanislav Cherchesov             Unai Emery
1             Andrey Kobelev          Rustem Khuzin
2             Leonid Slutski           Dan Petrescu
3               Slaven Bilic         Kurban Berdyev
4             Andrey Kobelev        Gadzhi Gadzhiev
...                      ...                    ...
65722    Ruud van Nistelrooy          Rogier Meijer
65723           Ernest Faber           Phillip Cocu
65724           Phillip Cocu          Peter Hyballa
65725          Rogier Meijer    Ruud van Nistelrooy
65726             Peter Bosz          Rogier Meijer

[52062 rows x 2 columns]


In [ ]:
#nella tabella GAME_EVENTS inserisco i valori mancanti nel player_in_id
for index, game_e in df_game_events.iterrows():
    if pd.isnull(game_e["player_in_id"]):
        df_game_events.at[index, "player_in_id"] = 0

In [ ]:
#nella tabella GAME_EVENTS inserisco i valori mancanti nel player_assist_id
for index, game_e in df_game_events.iterrows():
    if pd.isnull(game_e["player_assist_id"]):
        df_game_events.at[index, "player_assist_id"] = 0
 
#conversione degli id in interi
df_game_events['player_in_id'] = df_game_events['player_in_id'].astype(int)
df_game_events['player_assist_id'] = df_game_events['player_assist_id'].astype(int)
df_game_events["date"] = pd.to_datetime(df_game_events["date"])
df_game_events = df_game_events[df_game_events['game_id'].isin(df_games['game_id'])]

In [ ]:
#CONTROLLO SE GLI ID DEI CLUB SU CLUB_GAMES CI SONO IN CLUB ALTRIMENTI LI ESCLUDO
# Ottenere i 'club_id' di df_club_games solo quando 'own_manager_name' ha valori nulli
valori_colonna1_con_null = df_club_games.loc[df_club_games['own_manager_name'].isnull(), 'club_id']
valori_colonna2_con_null = df_club_games.loc[df_club_games['opponent_manager_name'].isnull(), 'club_id']
 
# Verificare se i 'club_id' ottenuti sono presenti in 'df_clubs'
valori_presenti_in_df2 = valori_colonna1_con_null.isin(df_clubs['club_id'])
valori_presenti_in_df22 = valori_colonna2_con_null.isin(df_clubs['club_id'])
 
valori_false = valori_colonna1_con_null[~valori_presenti_in_df2]
valori_false2 = valori_colonna2_con_null[~valori_presenti_in_df22]
 
# Itera attraverso gli elementi (indice, valore) di valori_false
rows_to_drop = set()  # Utilizziamo un set per evitare duplicati
 
for index, c_game in df_club_games.iterrows():
    for indice, valore in valori_false.items():
        if c_game["club_id"] == indice:
            rows_to_drop.add(index)
 
    for indice2, valore2 in valori_false2.items():
        if c_game["club_id"] == indice2:
            rows_to_drop.add(index)
 
# Converti il set in una lista
rows_to_drop = list(rows_to_drop)
 
# Elimina le righe dopo aver completato il ciclo annidato
df_club_games.drop(rows_to_drop, inplace=True)

In [ ]:
#VADO A PRENDERE I NOMI DEGLI ALLENATORI DA GAMES E LI METTO IN CLUB_GAMES PER LE SQUADRE
for index, club_g in df_club_games.iterrows():
    if (pd.isnull(club_g['own_manager_name']) and pd.isnull(club_g['opponent_manager_name'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            club_info_2 = df_games[df_games['away_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_manager_name'] = manager_info.iloc[0]['home_club_manager_name']
            if not club_info_2.empty:
                df_club_games.at[index, 'opponent_manager_name'] = manager_info.iloc[0]['away_club_manager_name']
            else:
                print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)
    elif (pd.isnull(club_g['own_manager_name']) or pd.isnull(club_g['opponent_manager_name'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_manager_name'] = manager_info.iloc[0]['home_club_manager_name']
            else:
                club_info_2 = df_games[df_games['away_club_id'] == club_id]
                if not club_info_2.empty:
                    df_club_games.at[index, 'opponent_manager_name'] = manager_info.iloc[0]['away_club_manager_name']
                else:
                    print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)

In [ ]:
#VADO A PRENDERE LE POSIZIONI DELLE SQUADRE DA GAMES E LI METTO IN CLUB_GAMES
for index, club_g in df_club_games.iterrows():
    if (pd.isnull(club_g['own_position']) and pd.isnull(club_g['opponent_position'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            club_info_2 = df_games[df_games['away_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_position'] = manager_info.iloc[0]['home_club_position']
            if not club_info_2.empty:
                df_club_games.at[index, 'opponent_position'] = manager_info.iloc[0]['away_club_position']
            else:
                print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)
    elif (pd.isnull(club_g['own_manager_name']) or pd.isnull(club_g['opponent_manager_name'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_position'] = manager_info.iloc[0]['home_club_position']
            else:
                club_info_2 = df_games[df_games['away_club_id'] == club_id]
                if not club_info_2.empty:
                    df_club_games.at[index, 'opponent_position'] = manager_info.iloc[0]['away_club_position']
                else:
                    print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)

In [ ]:
#INSERISCO I NOMI DEGLI ALLENATORI NELLA TABELLA CLUB PRENDENDOLI DA GAME
for index, club in df_clubs.iterrows():
    if (pd.isnull(club['coach_name'])):
        club_id = club['club_id']
        manager_info = df_games[df_games['home_club_id'] == club_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            if not club_info.empty:
                df_clubs.at[index, 'coach_name'] = manager_info.iloc[0]['home_club_manager_name']

In [ ]:
#dove manca average_age nella tabella CLUB mettiamo la media che è 25.7
for index, club in df_clubs.iterrows():
    if pd.isnull(club["average_age"]):
        df_clubs.at[index, "average_age"] = 25.7

In [ ]:
#da CLUBS posso eliminare la percentuale e il numero di stranieri e il valore totale della squadra perché inutile
elimino = ["foreigners_number", "foreigners_percentage", "total_market_value"]
df_clubs.drop(columns = elimino, inplace = True)

In [ ]:
#dove manca contry_name nella tabella COMPETITION mettiamo Europa
for index, competition in df_competitions.iterrows():
    if pd.isnull(competition["country_name"]):
        df_competitions.at[index, "country_name"] = "Europe"

In [ ]:
#dove manca domestic_league_code nella tabella COMPETITION mettiamo EU
for index, competition in df_competitions.iterrows():
    if pd.isnull(competition["domestic_league_code"]):
        df_competitions.at[index, "domestic_league_code"] = "EU"

In [ ]:
#Quando non c'è il player_name in APPEARANCES elimino la riga perché tanto il player non ci sta nella tab player
for index, app in df_appearances.iterrows():
    if pd.isnull(app["player_name"]):
        df_appearances.drop(index, inplace=True)

In [ ]:
#creo i csv a fine etl
df_players.to_csv('../csv_post_etl/players_post.csv', index=False)
df_games.to_csv('../csv_post_etl/games_post.csv', index=False)
df_game_lineups.to_csv('../csv_post_etl/game_lineups_post.csv', index=False)
df_appearances.to_csv('../csv_post_etl/appearances_post.csv', index=False)
df_club_games.to_csv('../csv_post_etl/club_games_post.csv', index=False)
df_clubs.to_csv('../csv_post_etl/clubs_post.csv', index=False)
df_game_events.to_csv('../csv_post_etl/game_events_post.csv', index=False)
df_player_valuations.to_csv('../csv_post_etl/player_valuations_post.csv', index=False)
df_competitions.to_csv('../csv_post_etl/competitions_post.csv', index=False)

In [ ]:
#questo crea uno script in python nella cartella codice_py in modo che sia importabile (quando creeremo i nuovi csv pero sarà inutile poichè lavoreremo su quelli direttamente)
!jupyter nbconvert --to script --output-dir codice_py etl.ipynb


[NbConvertApp] Converting notebook etl.ipynb to script
[NbConvertApp] Writing 9766 bytes to codice_py\etl.py
